In [2]:
import kagglehub
import os

/opt/anaconda3/envs/ml_ai/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Download latest version
data_path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")
data_path = os.path.join(data_path, 'chest_xray')

In [8]:
! python cnn.py -data_dir {data_path} -v

Number of classes    :  2
Shape of image       :  torch.Size([1, 256, 256])
Train batches        :  652
No. of train images  :  5216
No. of valid images  :  16
Labels               :  {'NORMAL': 0, 'PNEUMONIA': 1}
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 128, 64, 64]           2,176
         LayerNorm-2          [-1, 128, 64, 64]       1,048,576
            Conv2d-3          [-1, 128, 64, 64]         802,944
         LayerNorm-4          [-1, 128, 64, 64]       1,048,576
            Conv2d-5          [-1, 256, 64, 64]          33,024
              GELU-6          [-1, 256, 64, 64]               0
            Conv2d-7          [-1, 128, 64, 64]          32,896
         AvgPool2d-8          [-1, 128, 32, 32]               0
           Flatten-9               [-1, 131072]               0
           Linear-10                    [-1, 2]         262,146
Total params: 3,2